# Day 1

## part 1

In [4]:
import re

In [5]:
pattern = re.compile(r"\d")
calibration_sum = 0

with open("inputs/01.txt") as f:
    for line in f:
        nums = pattern.findall(line)
        calibration_sum += int(f"{nums[0]}{nums[-1]}")

calibration_sum

CPU times: user 1.99 ms, sys: 1.83 ms, total: 3.83 ms
Wall time: 3.08 ms


54927

## part 2

In [6]:
spelled_numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}

# handle overlapping matches like "eighthree"
pattern = re.compile(f"(?=(\d|{'|'.join(spelled_numbers.keys())}))")
calibration_sum = 0

with open("inputs/01.txt") as f:
    for line in f:
        nums = [spelled_numbers.get(x, x) for x in pattern.findall(line)]
        calibration_sum += int(f"{nums[0]}{nums[-1]}")

calibration_sum

CPU times: user 3.29 ms, sys: 1.09 ms, total: 4.39 ms
Wall time: 3.46 ms


54581

# Day 2

## part 1

In [ ]:
import re


max_cubes = {
    "red": 12,
    "green": 13,
    "blue": 14
}

pattern = re.compile(r"(\d+) (red|green|blue)")
valid_ids_sum = 0

with open("inputs/02.txt") as f:
    for line in f:
        id, trials = line.split(":")
        id = int(id.split()[-1])
        
        trials = pattern.findall(trials)
        trials = [(color, int(num)) for num, color in trials]
        valid = [max_cubes[color] >= num for (color, num) in trials]
        
        if all(valid): valid_ids_sum += id
    
valid_ids_sum

## part 2

In [9]:
import re
import math


pattern = re.compile(r"(\d+) (red|green|blue)")
power_set_sum = 0

with open("inputs/02.txt") as f:
    for line in f:
        trials = line.split(":")[-1]
        trials = pattern.findall(trials)
        trials = [(color, int(num)) for num, color in trials]

        max_color = {"red": 0, "green": 0, "blue": 0}
        
        for color, num in trials:
            max_color[color] = max(max_color[color], num)

        power_set_sum += math.prod(max_color.values())

power_set_sum

67335

# Day 3

## part 1

In [62]:
import string
import itertools



def padding_row(line_len):
    return "".join(["."] * line_len + ["\n"])


def pad_row(row):
    return [".", *row.strip(), "."]


def pad_iterator(iterator):
    line_len = len(iterator.readline().strip())
    iterator.seek(0)

    return itertools.chain([padding_row(line_len)], iterator)


def part_number(top, middle, bottom, window_start, window_end, symbols):
    num = int("".join(middle[window_start:window_end]))
    
    top = set(top[window_start - 1 : window_end + 1])
    middle = set(middle[window_start - 1 : window_end + 1])
    bottom = set(bottom[window_start - 1 : window_end + 1])

    if len((top | middle | bottom) & symbols) > 0:
        return num
    else:
        return 0
    

symbols = set(string.punctuation) - {"."}
is_last_row = False
parts_sum = 0

with open("inputs/03.txt") as f:
    f = pad_iterator(f)
    
    top = next(f)
    middle = next(f)
    bottom = next(f)
    
    top, middle = pad_row(top), pad_row(middle)
    
    while bottom is not None:
        bottom = pad_row(bottom)
        window_start = window_end = 1
        found_number = False

        while window_end < len(middle):
            match found_number, middle[window_end].isnumeric():
                case True, False:
                    parts_sum += part_number(top, middle, bottom, window_start, window_end, symbols)
                    found_number = False
                case False, True:
                    found_number = True
                    window_start = window_end

            window_end += 1

        top = middle
        middle = bottom
        bottom = next(f, None)
        
        if bottom is None and not is_last_row:
            is_last_row = True
            bottom = padding_row(len(middle))

parts_sum

525911

## part 2

In [99]:
import math
import itertools


def padding_row(line_len):
    return "".join(["."] * line_len + ["\n"])


def pad_row(row):
    return [".", *row.strip(), "."]


def pad_iterator(iterator):
    line_len = len(iterator.readline().strip())
    iterator.seek(0)

    return itertools.chain([padding_row(line_len)], iterator)

def get_number(row, position):
    left = right = position
    while row[left - 1].isnumeric():
        left -= 1
    while row[right + 1].isnumeric():
        right += 1
    return int("".join(row[left:right + 1]))


def check_top_or_bottom(row, position):
    nums = []
    if row[position].isnumeric():
        nums.append(get_number(row, position))
    else:
        if row[position - 1].isnumeric():
            nums.append(get_number(row, position - 1))
        if row[position + 1].isnumeric():
            nums.append(get_number(row, position  + 1))
    
    return nums


def check_left_or_right(row, position):
    nums = []
    if row[position].isnumeric():
        nums.append(get_number(row, position))
    
    return nums


is_last_row = False
gear_product = 0

with open("inputs/03.txt") as f:
    f = pad_iterator(f)
    
    top = next(f)
    middle = next(f)
    bottom = next(f)
    
    top, middle = pad_row(top), pad_row(middle)
        
    while bottom is not None:
        bottom = pad_row(bottom)
        
        for position, char in enumerate(middle):
            if char == "*":
                nums = [
                    *check_top_or_bottom(top, position),
                    *check_top_or_bottom(bottom, position),
                    *check_left_or_right(middle, position - 1),
                    *check_left_or_right(middle, position + 1)
                ]
                
                if len(nums) == 2:
                    gear_product += math.prod(nums)
        
        top = middle
        middle = bottom
        bottom = next(f, None)
        
        if bottom is None and not is_last_row:
            is_last_row = True
            bottom = padding_row(len(middle))

gear_product

75805607

# Day 4

In [110]:
def get_deck(filepath):
    deck = []
    with open(filepath) as f:
        for line in f:
            winning, played = get_plays(line)
            deck.append((winning, played))
            
    return deck


def get_plays(line):
    nums = line.split(":")[-1]
    winning, played = nums.split("|")
    winning, played = set(map(int, winning.split())), set(map(int, played.split()))
    
    return winning, played


def get_num_matches(winning, played):
    return len(winning & played)

## part 1

In [111]:
points = 0
deck = get_deck("inputs/04.txt")

for winning, played in deck:
    num_matches = get_num_matches(winning, played)
    if num_matches > 0: points += 2**(num_matches - 1)

points

23847

## part 2

In [112]:
deck = get_deck("inputs/04.txt")
weights = [1]*len(deck)
i = 0

for i, (winning, played) in enumerate(deck):
    num_matches = get_num_matches(winning, played)
    
    if num_matches > 0:
        for j in range(1, num_matches + 1):
            weights[i + j] += weights[i]

sum(weights)

8570000